In [1]:
#####Hierarchical structured latent factor model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import seaborn as sns
import gc
from scipy import sparse
from scipy.stats import norm
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize

#np.random.seed(98537)

In [2]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no, pattern):
    z_id = np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1)
    return z_id
    if pattern==1:
        Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
        return z_id, Z

In [3]:
####データの発生####
##データの設定
#要素ごとのユニーク数を設定
k = 10
hh = 3000
item = 22500
tag = 300
category = 150
max_m = 20
k_vec = np.repeat(1, k)
m_vec = np.repeat(1, max_m)

#要素ごとの頻度を生成
pt = np.random.poisson(np.random.gamma(27.5, 1/0.2, hh))
N = np.sum(pt)
m = np.random.poisson(7.5, N)
m[m==0] = 1; m[m >= max_m] = max_m
max_m = np.max(m)
M = np.sum(m)

In [4]:
##idとインデックスの設定
#idの設定
d_id = np.repeat(np.arange(hh), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(hh)])))
m_id = np.repeat(np.arange(N), m)

#インデックスの設定
freq_m = np.repeat(0, hh)
d_index = [i for i in range(hh)]
m_index = [i for i in range(N)]
for i in range(hh):
    d_index[i] = np.array(np.where(d_id==i)[0], dtype="int")
    freq_m[i] = np.sum(m[d_index[i]])
for i in range(N):
    if i==0:
        m_index[i] = np.arange(m[0])
    else:
        m_index[i] = np.max(m_index[i-1]) + np.arange(m[i]) + 1
freq_id = np.repeat(np.arange(hh), freq_m)

In [ ]:
##アイテムの生成
#多項分布からトピックを生成
topic = 30
theta_topic = np.random.dirichlet(np.repeat(0.2, topic), hh)
phi_topic = np.random.dirichlet(np.repeat(0.5, item), topic)
z = np.array(rmnom(theta_topic[np.repeat(d_id, m), ], M, topic, np.arange(M), 0), dtype="int")

#トピックからアイテムを生成
item_id = np.repeat(0, M)
item_dt = np.full((N, np.max(m)), item, dtype="int16")
for i in range(N):
    if i%100000==0:
        print(i)
    index = m_index[i]
    target_z = z[index]
    while True:
        temp_id = np.array(rmnom(phi_topic[target_z, ], m[i], item, np.arange(m[i]), 0), dtype="int16")
        if np.unique(temp_id).shape[0]==m[i]:
            item_id[index] = temp_id
            item_dt[i, np.arange(m[i])] = temp_id
            break

0


In [ ]:
##アイテムにカテゴリを割り当て
#多項分布からトピックを生成
topic = 20
phi_tag = np.random.dirichlet(np.repeat(0.5, tag), topic)
phi_category = np.random.dirichlet(np.repeat(0.5, category), topic)
z = np.dot(np.random.multinomial(1, np.random.dirichlet(np.repeat(1.0, topic)), item), np.arange(topic))

#トピックからカテゴリを割り当て
tag_id = np.array(rmnom(phi_tag[z, ], item, tag, np.arange(item), 0), dtype="int16")
category_id = np.array(rmnom(phi_category[z, ], item, category, np.arange(item), 0), dtype="int16")

In [ ]:
##インデックスを設定
#変数割当のインデックス
allocation_index = [j for j in range(max_m)]
allocation_item = [j for j in range(max_m)]
for j in range(max_m):
    allocation_index[j] = np.array(np.where(item_dt[:, j]!=item)[0], dtype="int")
    allocation_item[j] = item_dt[allocation_index[j], j]
    
#アイテムのインデックス
item_index = [i for i in range(item)]
item_n = np.repeat(0, item)
for i in range(item):
    item_index[i] = np.array(np.where(np.sum(item_dt==i, axis=1)==1)[0], dtype="int")
    item_n[i] = item_index[i].shape[0]
    
#タグとカテゴリーのインデックス
tag_index = [j for j in range(tag)]
category_index = [j for j in range(category)]
tag_n = np.repeat(0, tag)
category_n = np.repeat(0, category)
for j in range(tag):
    tag_index[j] = np.array(np.where(tag_id==j)[0], dtype="int")
    tag_n[j] = tag_index[j].shape[0]
for j in range(category):
    category_index[j] = np.array(np.where(category_id==j)[0], dtype="int")
    category_n[j] = category_index[j].shape[0]

In [ ]:
##応答変数を生成
rp = 0
while True:
    rp = rp + 1
    
    #タグとカテゴリーの階層モデルのパラメータを生成
    alpha_g = np.repeat(0, k)
    alpha_c = np.repeat(0, k)
    tau_g = np.diag(np.repeat(0.25, k))
    tau_c = np.diag(np.repeat(0.25, k))
    alphat_g = alpha_g.copy(); alphat_c = alpha_c.copy()
    taut_g = tau_g.copy(); taut_c = tau_c.copy()

    #ユーザーとアイテムの階層モデルのパラメータを生成
    alpha_u = np.repeat(0, k)
    alpha_v = np.random.normal(0, 0.5, k*k).reshape(k, k)
    theta_g = np.random.multivariate_normal(alpha_g, tau_g, tag)
    theta_c = np.random.multivariate_normal(alpha_c, tau_c, category)
    theta_gc = np.dot(theta_g[tag_id, ] * theta_c[category_id, ], alpha_v)
    tau_u = np.array([0.4])
    Cov_u = np.diag(np.repeat(0.5, k))
    Cov_v = np.diag(np.repeat(0.2, k))
    alphat_u = alpha_u.copy(); alphat_v = alpha_v.copy(); thetat_g = theta_g.copy(); thetat_c = theta_c.copy()
    taut_u = tau_u.copy(); Covt_u = Cov_u.copy(); Covt_v = Cov_v

    #モデルパラメータを生成
    Sigma = np.array([1.0])
    beta = np.array([-1.0])
    beta_u = np.random.normal(0, tau_u, hh)
    theta_u = np.random.multivariate_normal(alpha_u, Cov_u, hh)
    theta_v = np.zeros((item, k))
    for i in range(item):
        theta_v[i, ] = np.random.multivariate_normal(theta_gc[i, ], Cov_v, 1)
    betat = beta.copy(); betat_u = beta_u.copy()
    thetat_u = theta_u.copy(); thetat_v = theta_v.copy()
        
    #モデルの期待値
    theta_item = np.zeros((N, k))
    for j in range(max_m):
        index = allocation_index[j]
        theta_item[index, ] += theta_v[item_dt[index, j], ]
    uv = np.sum(theta_u[d_id, ] * theta_item, axis=1)
    mu = beta + beta_u[d_id] + uv

    #潜在効用から二値変数を生成
    U = np.random.normal(mu, Sigma, N)
    y = np.array(U > 0, dtype="int")

    #break条件
    print([rp, np.round(np.mean(y), 3)])
    if (np.mean(y) >0.2) & (np.mean(y) < 0.4):
        break

In [ ]:
#潜在変数ごとのヒストグラム
dt = [U, mu, beta_u, uv]
fig_range = np.append(np.array([np.min(U), np.min(mu), np.min(beta_u), np.min(uv)]), 
                      np.array([np.max(U), np.max(mu), np.max(beta_u), np.max(uv)])).reshape(2, len(dt))
colorlist = ["r", "g", "b", "c", "m", "y", "k", "w"]
legend = ["潜在効用の分布", "モデル期待値の分布", "ユーザーランダム効果の分布", "uvの分布"]
fig = plt.figure(figsize=(8.0, 5.0))
for j in range(len(dt)):
    ax = fig.add_subplot(2, 2, j+1)
    ax.hist(dt[j],  bins=25, range=(fig_range[0, j], fig_range[1, j]), color=colorlist[j])
    plt.title(legend[j], fontsize=12.5)
fig.tight_layout()
plt.show()

In [ ]:
####テストデータの生成####
##データの設定
pt0 = np.random.poisson(np.random.gamma(10.0, 1/0.25, hh))
N0 = np.sum(pt0)
m0 = np.random.poisson(5.5, N0)
m0[m0==0] = 1; m0[m0 >= max_m] = max_m
M0 = np.sum(m0)

In [ ]:
##idとインデックスの設定
#idの設定
d_id0 = np.repeat(np.arange(hh), pt0)
pt_id0 = np.array(list(itertools.chain(*[np.array(range(pt0[i]), dtype="int") for i in range(hh)])))
m_id0 = np.repeat(np.arange(N0), m0)

#インデックスの設定
freq_m0 = np.repeat(0, hh)
d_index0 = [i for i in range(hh)]
m_index0 = [i for i in range(N0)]
for i in range(hh):
    d_index0[i] = np.array(np.where(d_id0==i)[0], dtype="int")
    freq_m0[i] = np.sum(m[d_index0[i]])
for i in range(N0):
    if i==0:
        m_index0[i] = np.arange(m0[0])
    else:
        m_index0[i] = np.max(m_index0[i-1]) + np.arange(m0[i]) + 1
freq_id0 = np.repeat(np.arange(hh), freq_m0)

In [ ]:
##アイテムの生成
#多項分布からトピックを生成
z = np.array(rmnom(theta_topic[np.repeat(d_id0, m0), ], M0, topic, np.arange(M0), 0), dtype="int")

#トピックからアイテムを生成
item_id0 = np.repeat(0, M0)
item_dt0 = np.full((N0, max_m), item, dtype="int16")
for i in range(N0):
    if i%100000==0:
        print(i)
    index = m_index0[i]
    target_z = z[index]
    while True:
        temp_id = np.array(rmnom(phi_topic[target_z, ], m0[i], item, np.arange(m0[i]), 0), dtype="int16")
        if np.unique(temp_id).shape[0]==m0[i]:
            item_id0[index] = temp_id
            item_dt0[i, np.arange(m0[i])] = temp_id
            break

In [ ]:
##インデックスを設定
#変数割当のインデックス
allocation_index0 = [j for j in range(max_m)]
allocation_item0 = [j for j in range(max_m)]
for j in range(max_m):
    allocation_index0[j] = np.array(np.where(item_dt0[:, j]!=item)[0], dtype="int")
    allocation_item0[j] = item_dt0[allocation_index0[j], j]

#アイテムのインデックス
item_index0 = [i for i in range(item)]
item_n0 = np.repeat(0, item)
for i in range(item):
    item_index0[i] = np.array(np.where(np.sum(item_dt0==i, axis=1)==1)[0], dtype="int")
    item_n0[i] = item_index0[i].shape[0]

In [ ]:
##応答変数を生成
#モデルの期待値
theta_item = np.zeros((N0, k))
for j in range(max_m):
    index = allocation_index0[j]
    theta_item[index, ] += theta_v[item_dt0[index, j], ]
uv = np.sum(theta_u[d_id0, ] * theta_item, axis=1)
mu = beta + beta_u[d_id0] + uv

#潜在効用から二値変数を生成
U0 = np.random.normal(mu, Sigma, N0)
y0 = np.array(U0 > 0, dtype="int")

In [ ]:
####マルコフ連鎖モンテカルロ法でパラメータを推定####
##切断正規分布の乱数を発生させる関数
def rtnorm(mu, sigma, a, b, n):
    FA = norm.cdf(a, mu, sigma)
    FB = norm.cdf(b, mu, sigma)
    return norm.ppf(np.random.uniform(0, 1, n)*(FB-FA)+FA, mu, sigma)

In [ ]:
##ベイジアン多変量回帰モデルをギブスサンプリングする関数
def rmultireg(Y, X, inv_XXV, XY, Cov, ADelta, Deltabar, V, nu, n, col, k):
    #事後分布のパラメータを設定
    beta_mu = np.dot(inv_XXV, XY + np.dot(ADelta, Deltabar)).T.reshape(-1)   #平均ベクトル
    sigma = np.kron(Cov, inv_XXV)   #分散共分散行列

    #パラメータをサンプリング
    beta_vec = np.random.multivariate_normal(beta_mu, sigma, 1)
    beta = beta_vec.reshape(col, k, order='F')   #回帰行列に変換

    
    ##逆ウィシャート分布から分散共分散行列をサンプリング
    #モデル誤差を設定
    mu = np.dot(X, beta)
    er = Y - mu

    #逆ウィシャート分布のパラメータ
    IW_R = np.dot(er.T, er) + V
    Sn = n + nu

    #パラメータをサンプリング
    Cov = scipy.stats.invwishart.rvs(Sn, IW_R, 1)
    return beta, Cov

In [ ]:
##アルゴリズムの設定
k = 10
R = 2000
keep = 10
burnin = int(100/keep)
iter = 0
disp = 10

In [ ]:
##事前分布の設定
#階層モデルの事前分布
delta = np.repeat(0, k)
nu = 1
V = 0.1 * np.diag(np.ones(k))
s01 = 0.1
s01 = 0.1

#モデルパラメータの事前分布
tau = 100 
s02 = 1.0
v02 = 1.0

In [ ]:
##パラメータの真値
#タグとカテゴリーの階層モデルのパラメータを生成
alpha_g = alphat_g.copy()
alpha_c = alphat_c.copy()
tau_g = taut_g.copy()
tau_c = taut_c.copy()
inv_tau_g = np.linalg.inv(tau_g)
inv_tau_c = np.linalg.inv(tau_c)

#ユーザーとアイテムの階層モデルのパラメータを生成
alpha_u = alphat_u.copy()
alpha_v = alphat_v.copy()
theta_g = thetat_g.copy()
theta_c = thetat_c.copy()
theta_gc = np.dot(theta_g[tag_id, ] * theta_c[category_id, ], alpha_v)
tau_u = taut_u.copy()
Cov_u = Covt_u.copy()
Cov_v = Covt_v.copy()
inv_Cov_u = np.linalg.inv(Cov_u)
inv_Cov_v = np.linalg.inv(Cov_v)

#モデルパラメータを生成
Sigma = np.array([1.0])
beta = betat.copy()
beta_u = betat_u.copy()
theta_u = thetat_u.copy()
theta_v = thetat_v.copy()

#モデルの期待値
theta_item = np.zeros((N, k))
for j in range(max_m):
    index = allocation_item[j]
    theta_item[allocation_index[j], ] += theta_v[index, ]
uv = np.sum(theta_u[d_id, ] * theta_item, axis=1)
mu = beta + beta_u[d_id] + uv

In [ ]:
##パラメータの初期値
#タグとカテゴリーの階層モデルのパラメータの初期値
alpha_g = np.repeat(0, k)
alpha_c = np.repeat(0, k)
tau_g = np.diag(np.repeat(1.0, k))
tau_c = np.diag(np.repeat(1.0, k))
inv_tau_g = np.linalg.inv(tau_g)
inv_tau_c = np.linalg.inv(tau_c)

#ユーザーとアイテムの階層モデルのパラメータの初期値
alpha_u = np.repeat(0, k)
alpha_v = np.random.normal(0, 0.5, k*k).reshape(k, k)
theta_g = np.random.multivariate_normal(alpha_g, np.diag(np.repeat(0.1, k)), tag)
theta_c = np.random.multivariate_normal(alpha_c, np.diag(np.repeat(0.1, k)), category)
theta_gc = np.dot(theta_g[tag_id, ] * theta_c[category_id], alpha_v)
tau_u = np.array([0.5])
Cov_u = np.diag(np.repeat(0.5, k))
Cov_v = np.diag(np.repeat(0.5, k))
inv_Cov_u = np.linalg.inv(Cov_u)
inv_Cov_v = np.linalg.inv(Cov_v)

#モデルパラメータを生成
Sigma = np.array([1.0])
beta = np.array([-1.0])
beta_u = np.random.normal(0, tau_u, hh)
theta_u = np.random.multivariate_normal(alpha_u, np.diag(np.repeat(0.01, k)) , hh)
theta_v = np.zeros((item, k))
for i in range(item):
    theta_v[i, ] = np.random.multivariate_normal(theta_gc[i, ], np.diag(np.repeat(0.01, k)), 1)
    
#モデルの期待値
theta_item = np.zeros((N, k))
for j in range(max_m):
    index = allocation_item[j]
    theta_item[allocation_index[j], ] += theta_v[index, ]
uv = np.sum(theta_u[d_id, ] * theta_item, axis=1)
mu = beta + beta_u[d_id] + uv

In [36]:
##データの設定
#切断領域を定義
a = (1-y)*(-100) + y*0
b = y*100 + (1-y)*0

In [37]:
##対数尤度の基準値
#学習データに対する1パラメータモデルの対数尤度
Prob = norm.cdf(np.mean(y), 0, Sigma)
LLst = np.sum(y*np.log(Prob)) + np.sum((1-y)*np.log(1-Prob))
print(LLst)

#学習データに対する真値での対数尤度
thetat_item = np.zeros((N, k))
for j in range(max_m):
    index = allocation_item[j]
    thetat_item[allocation_index[j], ] += thetat_v[index, ]
mut = betat + betat_u[d_id] + np.sum(thetat_u[d_id, ] * thetat_item, axis=1)
Prob = norm.cdf(mut, 0, Sigma)
Prob[Prob==1.0] = 0.9999999999; Prob[Prob==0] = np.power(10.0, -100)
LLbest = np.sum(y*np.log(Prob)) + np.sum((1-y)*np.log(1-Prob))
print(LLbest)


#テストデータに対する1パラメータモデルの対数尤度
Prob = norm.cdf(np.mean(y), 0, Sigma)
LLst0 = np.sum(y0*np.log(Prob)) + np.sum((1-y0)*np.log(1-Prob))
print(LLst0)

#テストデータに対する真値での対数尤度
thetat_item = np.zeros((N0, k))
for j in range(max_m):
    index = allocation_item0[j]
    thetat_item[allocation_index0[j], ] += thetat_v[index, ]
mut = betat + betat_u[d_id0] + np.sum(thetat_u[d_id0, ] * thetat_item, axis=1)
Prob = norm.cdf(mut, 0, Sigma)
Prob[Prob==1.0] = 0.9999999999; Prob[Prob==0] = np.power(10.0, -100)
LLbest0 = np.sum(y0*np.log(Prob)) + np.sum((1-y0)*np.log(1-Prob))
print(LLbest0)

-334027.60928900354
-80335.25697129106
-98930.21208865347
-26521.386678581875


In [38]:
####ギブスサンプリングでパラメータをサンプリング####
##切断正規分布から潜在効用を生成
#モデルの期待値
beta_vec = beta_u[d_id]; theta_u1 = theta_u[d_id, ]
theta_item = np.zeros((N, k))
for j in range(max_m):
    index = allocation_item[j]
    theta_item[allocation_index[j], ] += theta_v[index, ]
uv = np.sum(theta_u1 * theta_item, axis=1)
mu = beta + beta_vec + uv   #モデルの平均構造

#潜在効用を生成
U = rtnorm(mu, Sigma, a, b, N)

In [34]:
##モデルの期待値パラメータと標準偏差をサンプリング
#データの設定
Sigma_sq = np.power(Sigma, 2)
er_y = U - beta_vec - uv

#正規分布から期待値をサンプリング
weights = np.power(tau, 2) / (Sigma_sq/N + np.power(tau, 2))
mu_par = weights * np.mean(er_y)
beta = np.random.normal(mu_par, weights*Sigma_sq/N, 1)


##ユーザーのランダム効果をサンプリング
#モデル誤差を定義
er_y = U - beta - uv

#正規分布から事後分布をサンプリング
weights = np.repeat(0.0, hh)
mu_par = np.repeat(0.0, hh)
for i in range(hh):
    index = d_index[i]
    weights[i] = np.power(tau_u, 2) / (Sigma_sq/pt[i] + np.power(tau_u, 2))
    mu_par[i] = weights[i] * np.mean(er_y[index]) 
beta_u = np.random.normal(mu_par, weights*Sigma_sq/pt)
beta_vec = beta_u[d_id]

In [ ]:
##ユーザーの特徴ベクトルをサンプリング
#モデル誤差を定義
er_y = U - beta - beta_vec

for i in range(hh):
    #事後分布のパラメータ
    index = d_index[i]
    x1 = theta_item[index, ]; y1 = er_y[index]
    xy = np.dot(x1.T, y1)
    xxv = np.dot(x1.T, x1) + inv_Cov_u
    inv_xxv = np.linalg.inv(xxv)
    theta_par = np.dot(inv_xxv, xy + np.dot(inv_Cov_u, alpha_u))

    #多変量正規分布からパラメータをサンプリング
    theta_u[i, ] = np.random.multivariate_normal(theta_par, Sigma_sq*inv_xxv, 1)
theta_u1 = theta_u[d_id, ]

In [39]:
##アイテムの特徴ベクトルをサンプリング
for i in range(item):
    #事後分布のパラメータ
    index = item_index[i]
    x1 = theta_u1[index, ]; y1 = er_y[index] - uv[index] + np.sum(x1 * theta_v[i, ], axis=1)
    xy = np.dot(x1.T, y1)
    xxv = np.dot(x1.T, x1) + inv_Cov_v
    inv_xxv = np.linalg.inv(xxv)
    theta_par = np.dot(inv_xxv, xy + np.dot(inv_Cov_v, theta_gc[i, ]))

    #多変量正規分布からパラメータをサンプリング
    theta_v[i, ] = np.random.multivariate_normal(theta_par, Sigma_sq*inv_xxv, 1)

In [43]:
#モデルの期待値を更新
theta_item = np.zeros((N, k))
for j in range(max_m):
    index = allocation_item[j]
    theta_item[allocation_index[j], ] += theta_v[index, ]
uv = np.sum(theta_u1 * theta_item, axis=1)
mu = beta + beta_vec + uv

In [47]:
##階層モデルのパラメータをサンプリング
#ランダム効果の分散をサンプリング
er_u = beta_u - np.mean(beta_u)
s1 = np.sum(np.power(er_u, 2)) + s01
s2 = hh + s02
tau_u = np.sqrt(1/np.random.gamma(s2/2, 1/(s1/2), 1))

#ユーザーの特徴ベクトルの分散共分散行列をサンプリング
er_u = theta_u - np.mean(theta_u, axis=0)
IW_R = np.dot(er_u.T, er_u) + V
Sn = hh + nu
Cov_u = np.diag(np.diag(scipy.stats.invwishart.rvs(Sn, IW_R, 1)))
inv_Cov_u = np.linalg.inv(Cov_u)

In [66]:
##アイテムの特徴ベクトルの階層モデルのパラメータをサンプリング
#事後分布のパラメータ
j = 0
index = tag_index[j]
x1 = np.dot(theta_c[category_id[index], ], alpha_v); y1 = theta_v[index, ]
xy = np.dot(x1.T, y1)
xxv = np.dot(x1.T, x1) + inv_tau_g
inv_xxv = np.linalg.inv(xxv)
#np.dot(inv_xxv, xy + np.dot(inv_tau_g, alpha_g))


In [92]:
j = 0
index = tag_index[j]
x1 = thetat_c[np.repeat(category_id[index], k), ] * alphat_v.T[np.tile(np.arange(k), index.shape[0]), ]
y1 = thetat_v[index, ].reshape(-1)
np.dot(np.dot(np.linalg.inv(np.dot(x1.T, x1)), x1.T), y1)

array([-0.44291712, -0.39916201, -0.29132031,  0.91325116, -1.1981425 ,
       -0.15242933,  0.83733512, -0.11517956, -0.22901278,  0.05164763])

In [93]:
theta_g[j, ]

array([-0.54233546, -0.42599508, -0.25376023,  0.86748636, -1.2335253 ,
       -0.09709118,  0.84168545, -0.28822326, -0.21347253,  0.05344238])

In [73]:
theta_v[index, ].reshape(-1)

array([-1.74924714e+00,  7.33783620e-02,  2.74723200e-01,  4.21979650e-01,
        6.01579056e-01, -8.03186668e-03,  3.79268844e-01,  7.74917058e-02,
        5.24858735e-01,  1.02561884e+00, -1.62837649e+00, -7.36735192e-01,
        1.37002209e-01, -6.47611590e-01, -1.55540005e-01, -1.78933076e-01,
        6.55166304e-01,  1.06011783e+00, -4.19111168e-01,  8.69069193e-01,
       -4.34840130e-01,  5.93600237e-02,  6.21241504e-01,  5.26358891e-01,
       -3.61202954e-01, -4.39079589e-01,  7.86069691e-01,  1.80573298e-01,
        5.47055400e-01, -5.79457855e-02,  2.73064189e-01, -6.41269424e-01,
       -5.80905458e-01,  4.43343434e-01,  4.44036111e-02, -2.42806370e-01,
       -4.59485669e-01,  2.68993007e-01,  7.04394912e-01,  3.35419340e-01,
       -1.99427730e-01, -1.15012203e-01,  5.42545823e-01,  1.18720026e+00,
       -4.22485707e-01,  3.45226457e-01, -1.01645716e-02, -8.94103679e-01,
       -1.52500030e+00, -1.09105836e-01,  1.42825344e+00, -7.46487554e-01,
       -7.20641405e-01, -